In [ ]:
from requests import Session
from lxml import html
from datetime import datetime
import time
import random
import sys
import logging
import psycopg2
from psycopg2 import sql
import pytz
import ssl

In [ ]:

#rds settings
rds_host  = ""
name = "" 
db_name = "" 
password = ""


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")


def yelpScrapePage(business_id, 
                   page=0, # page
                   date_range=None # date range
                   ): 
    ''' 
    CAUTION: Do NOT use multi-threading to avoid getting blocked.
    '''
    status_code, results, keep_scraping = None, [], True
    
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start=" # add number

    with Session() as s:
        url = base_url + business_id + api_url + str(page*20)
        with s.get(url, timeout=5) as r:    
            status_code = r.status_code
            if status_code != 200:
                return status_code, results, keep_scraping

            response = dict(r.json()) 
            _html = html.fromstring(response['pagination'])
            text = _html.xpath("//div[@class='page-of-pages arrange_unit arrange_unit--fill']/text()")
            total_pages = int(text[0].strip().split(' ')[-1])
            if page+1 > total_pages:
                keep_scraping = False
                return status_code, results, keep_scraping
            
            _html = html.fromstring(response['review_list'])
            dates, stars, texts, review_ids, user_ids = [], [], [], [], []
            dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
            try:
                # datetime conversion causes problems for scraped data with blank dates
                dates = [datetime.strptime(d.strip(), format("%m/%d/%Y")) for d in dates]
                stars = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
                stars = [float(s.split(' ')[0]) for s in stars]
                texts = [e.text for e in _html.xpath("//div[@class='review-content']/p")]
                review_ids = _html.xpath("//div[@class='review review--with-sidebar']/@data-review-id")
                user_ids = [s.split(':')[1] for s in _html.xpath("//div[@class='review review--with-sidebar']/@data-signup-object")]
                results = [[date, star, text, review_id, user_id] 
                            for date, star, text, review_id, user_id 
                            in zip(dates, stars, texts, review_ids, user_ids)]
                # let blank dates pass through, resulting in all review dates 
                # not being formatted in datetime
            except ValueError:
                stars = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
                stars = [float(s.split(' ')[0]) for s in stars]
                texts = [e.text for e in _html.xpath("//div[@class='review-content']/p")]
                review_ids = _html.xpath("//div[@class='review review--with-sidebar']/@data-review-id")
                user_ids = [s.split(':')[1] for s in _html.xpath("//div[@class='review review--with-sidebar']/@data-signup-object")]
                results = [[date, star, text, review_id, user_id] 
                            for date, star, text, review_id, user_id 
                            in zip(dates, stars, texts, review_ids, user_ids)]
            
            # filter by date
            if date_range is not None:
                idx0, idx1 = None, None
                for i in range(len(dates)):
                    if dates[i]<=date_range[1]:
                        idx0 = i
                        break
                for i in range(len(dates)):
                    if dates[len(dates)-1-i]>=date_range[0]:
                        idx1 = len(dates)-1-i
                        break
                if idx0 is None or idx1 is None or idx1<idx0: 
                    results = []
                else:
                    results = results[idx0:idx1+1]
                    keep_scraping = False

    return status_code, results, keep_scraping


def fixMissingScrapedDates(results):
    for i in range(len(results)):
        try:
            if '/' not in results[i][0]:# if the date is missing
                try:
                    # assign the review date to be the same the 
                    # same date as the neighboring past review
                    results[i][0] = results[i+1][0]
                except IndexError:
                    try:
                        # if there isn't a past review, assign
                        # the date of a newer review
                        results[i][0] = results[i-1][0]
                    except IndexError:
                        # if there is only one other review, assign 
                        # the earliest review
                        results[i][0] = results[0][0]
        except TypeError:# if the date is not missing, do nothing
            pass
    return results
    
def dbConnect(results, business_id):
    item_count = 0
    with conn.cursor() as cur:
        for i in results:
            dateReview = i[0]
            try: 
                dateReview = datetime.strftime(dateReview, "%Y-%m-%d")
            except TypeError:
                dateReview = str(i[0])
                dateReview = dateReview.strip()
                dateReview = datetime.strptime(dateReview, "%m/%d/%Y")
            stars = float(i[1])
            reviewText = i[2]
            reviewId = i[3]
            userId = i[4]
            cur.execute("select distinct business_id from lab.yelp_scraping;")
            
            
            cur.execute(sql.SQL("insert into {} (review_id, business_id, user_id, stars, datetime, date, time, text, timestamp) values ( %s, %s, %s, %s, %s, %s, %s, %s, %s);")
                        .format(sql.Identifier('lab','yelp_scraping')),
                        [#'uuid-ossp',# auto generates new uuid
                         reviewId,# review id
                         business_id,
                         userId, # user id
                         stars, # stars
                         datetime.now(), #local time
                         dateReview,# date of review
                         datetime.now(),# time without timezone,
                         reviewText,# review text
                         datetime.now(pytz.utc),# time with timezone
                         ])

            conn.commit()
            cur.execute("select * from lab.yelp_scraping")
            for row in cur:
                item_count += 1
                logger.info(row)
        conn.commit()
    return "Added %d items from RDS postGreSQL table" %(item_count)

#normal dates
# business_id = "rR5Y9mp2Yob3rgetJscPWQ"

#missing date examples
business_id = "lLO8Nj-kPJ_b6vEs022GxQ"
business_id = "fLsXOkjewq5BqQbuUPYC9g"
results = yelpScrapePage(business_id)
results = fixMissingScrapedDates(results[1])
dbConnect(results, business_id)